# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [4]:
a = rand(10^7) # array of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.646849  
 0.970819  
 0.399479  
 0.708606  
 0.36707   
 0.0702605 
 0.26571   
 0.672445  
 0.317429  
 0.00268663
 0.145896  
 0.570564  
 0.192124  
 ⋮         
 0.217784  
 0.617378  
 0.468259  
 0.178476  
 0.0654431 
 0.0781863 
 0.16207   
 0.619255  
 0.281447  
 0.107603  
 0.388685  
 0.326295  

In [5]:
sum(a) # one expects this is 10^7 * .5 , since the mean of each entry is .5

4.999829528461973e6

# Benchmarking a few ways in a few languages

In [6]:
using BenchmarkTools  # Julia package for benchmarking

#  1. The C language: (8.1 msecs)

C is fften considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

You can put C code in a Julia session, compile it, and run it.

In [7]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [8]:
c_sum(a)

4.999829528461543e6

In [9]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

We can now benchmark the C code directly from Julia:

In [10]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.387 ms (0.00% GC)
  median time:      9.682 ms (0.00% GC)
  mean time:        10.172 ms (0.00% GC)
  maximum time:     15.703 ms (0.00% GC)
  --------------
  samples:          490
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [11]:
println("C: Fastest time was $(minimum(c_bench.times)/1e6) msecs.")

C: Fastest time was 9.387176 msecs.


# 2. Python's built in `sum` (68 msecs)

In [12]:
# Julia interface to Python:
using PyCall

INFO: Precompiling module PyCall.


In [13]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [14]:
pysum(a)

4.999829528461543e6

In [15]:
pysum(a) ≈ sum(a)

true

In [16]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     84.496 ms (0.00% GC)
  median time:      87.481 ms (0.00% GC)
  mean time:        89.284 ms (0.00% GC)
  maximum time:     108.453 ms (0.00% GC)
  --------------
  samples:          56
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [17]:
println("Python (built in): fastest time was $(minimum(py_list_bench.times)/1e6) msecs.")

Python (built in): fastest time was 84.496336 msecs.


# 3. Python: `numpy` (3.9 msec)  

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python

If it is not installed, install it from Julia as follows:

In [18]:
# using Conda 
# Conda.add("numpy")

In [19]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  960 bytes
  allocs estimate:  25
  --------------
  minimum time:     5.462 ms (0.00% GC)
  median time:      5.641 ms (0.00% GC)
  mean time:        5.955 ms (0.00% GC)
  maximum time:     10.661 ms (0.00% GC)
  --------------
  samples:          834
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [20]:
numpy_sum(apy_list) # python thing

4.999829528461976e6

In [21]:
numpy_sum(apy_list) ≈ sum(a)

true

# 4. Python, hand written (419 msec!)

In [22]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
mysum_py = py"""mysum"""o

PyObject <function mysum at 0x33f0e48c0>

In [23]:
@benchmark $mysum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     521.158 ms (0.00% GC)
  median time:      547.453 ms (0.00% GC)
  mean time:        546.987 ms (0.00% GC)
  maximum time:     572.114 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [24]:
mysum_py(apy_list)

4.999829528461543e6

In [25]:
mysum_py(apy_list) ≈ sum(a)

true

# 5. Julia (built-in) (3.7 msec) 

## Written directly in Julia, not in C!

In [26]:
@which sum(a)

sum(a) at reduce.jl:235

In [27]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.448 ms (0.00% GC)
  median time:      5.749 ms (0.00% GC)
  mean time:        6.018 ms (0.00% GC)
  maximum time:     9.156 ms (0.00% GC)
  --------------
  samples:          825
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

# 6. Julia (hand-written) (8.1 msec, same as hand-written C)

In [28]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [29]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.799 ms (0.00% GC)
  median time:      10.051 ms (0.00% GC)
  mean time:        10.283 ms (0.00% GC)
  maximum time:     19.687 ms (0.00% GC)
  --------------
  samples:          485
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%